### Mencoba membuat RAG lokal dengan llama2 70B diatas laptop core I5.

Kita buat file .env untuk simpan credential API dll selanjutnya declare model, tak hanya untuk llama2 tapi juga model OpenAI (melalui API) sebagai pembanding output <br>

In [1]:
import os
from dotenv import load_dotenv

MODEL = "llama2"

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_KEY")
# MODEL = "gpt-3.5-turbo"

# MODEL = "mixtral:8x7b"


In [2]:
#from langchain_openai.chat_models import ChatOpenAI
# from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings


# model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)
model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"Sure, here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope that made you smile!"

In [3]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

'\nWhy was the math book sad?\n\nBecause it had too many problems!'

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t \nanswer the question, reply "I don\'t know".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
chain = prompt | model | parser

chain.invoke({"context": "My parents named me Jhony Iskandar", "question": "What's your name'?"})

' Sure! Based on the context provided, the answer to the question "What\'s your name?" would be "Jhony Iskandar."'

In [6]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("./pengumuman.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': './pengumuman.pdf', 'page': 0}, page_content='P  E  N  G  U  M  U  M  A  N \nNOMOR :  800.1.2.6/   02  /PANSEL -JPT/20 24 \n \nTENTANG  \nSELEKSI TERBUKA PENGISIAN J ABATAN PIMPINAN TINGGI PRATAMA  \n DI LINGKUNGAN PEMERINTAH KABUPATEN PROBOLINGGO  \nTAHUN 20 24 \n \nDalam rangka Pengisian Jabatan Pimpinan Tinggi Pratama di Lingkungan Pemerintah  \nKabupaten Probolinggo  Tahun 2024, sesuai Undang -Undang Nomor 20 Tahun 2023 tentang \nAparatur Sipil Negara, Peraturan Pemerintah Nomor 11 Tahun 2017 ten tang Manajemen Pegawai \nNegeri Sipil  sebagaimana telah diubah dengan Peraturan  Pemerintah Nomor 17 Tahun 2020, \nPeraturan Menteri  Pendayag unaan Aparatur Negara dan Reformasi Birokrasi  Nomor 15 Tahun 2019 \ntentang  Pengisian Jabatan Pimpinan Tinggi Pratama secara terbuka dan kompetitif di Lingkungan \nInstansi Pemerintah , maka Pemerintah Kabupaten Probolinggo membuka  kesempatan  kepada \nPegawai Negeri Sipil  di Lingkungan Pemerintah Kabupaten /Kota  

In [7]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

c:\Users\lenov\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [8]:
retriever = vectorstore.as_retriever()
retriever.invoke("pengumuman")

[Document(metadata={'source': './pengumuman.pdf', 'page': 2}, page_content='Pimpinan Ting gi Pratama di Lingkungan Pemerintah Kabupaten Probolinggo (Badan \nKepe gawaian dan Penge mbangan Sumber Daya Manusia Kabupaten Probolinggo) dengan \nalamat Jl. Raya Panglima Sud irman No. 134 , Lantai 2 Kantor Bupati Proboling go dimulai \ntanggal 22 Januari  2024 sampai dengan 5 Febr uari 2024 pukul 0 8.00 s.d.16.00 WIB ;  \n3. Dokumen persyaratan yang  harus dilengkapi sebagai berikut:  \na. Surat lamaran diketik komputer dan dita ndatangani di atas materai tempel Rp. 10.000 ,-  \n(format terlampir ); \nb. Daftar Riwayat Hidup (format terlampir); \nc. Foto berwarna terbaru ukuran 4 x 6 cm dengan latar belakang me rah sebanyak 2 (dua) \nlembar ; \nd. Foto copy Kartu Tanda Penduduk;  \ne. Foto copy SK Kenaikan Pangkat terakhir;  \nf. Foto copy SK Jabatan  (mulai pengangkatan PNS samp ai dengan jab atan terakhir ); \ng. Foto copy SK Pengangkatan pertama kali dalam Jabatan A dministr ator (Eselon I